# Misión 4: Evaluando y mejorando el robot bróker

In [ ]:
"""

Las diferencias en los profits obtenidos por los agentes en relacion al uso de diferentes arquitecturas pueden atribuirse a varias razones,
incluyendo la complejidad del modelo, la capacidad de aprendizaje, y la calidad de los datos utilizados en el entrenamiento y prueba.

Aquí hay una conclusión de por qué cada agente pudo haber mostrado el profit observado:

dql_agent_exp1 (arquitectura simple) mostró un profit de 73.389:

Arquitectura Simple: Es posible que esta arquitectura, aunque simple, sea adecuada para el problema en cuestión.
Buen Ajuste al Entorno: Es posible que el agente haya aprendido a adaptarse bien al entorno y haya descubierto estrategias efectivas para maximizar el profit.
Esto podría deberse a una combinación de la capacidad del modelo y la exploración adecuada del espacio de acciones.
Buena Calidad de los Datos: Si los datos de prueba son representativos del entorno real y están bien etiquetados, el agente puede aprender de manera efectiva y generalizar sus conocimientos para tomar decisiones rentables.

dql_agent_exp2 (arquitectura simple) mostró un profit de 17.099:

Posible Sobreajuste: A pesar de tener una arquitectura simple, este agente puede haber sufrido de sobreajuste.
Exploración Insuficiente: Es posible que este agente no haya explorado adecuadamente el espacio de acciones durante el entrenamiento,
lo que podría haber llevado a subóptimos locales en lugar de estrategias más amplias y rentables.
Entorno Desafiante: Si el entorno es particularmente volátil o impredecible, un agente con una arquitectura simple puede tener dificultades para adaptarse y encontrar estrategias efectivas.

dql_agent_exp3 (arquitectura compleja) mostró un profit de 36.36:

Compromiso entre Complejidad y Generalización: Aunque esta arquitectura es más compleja que la primera,
puede haber encontrado un equilibrio entre la capacidad de aprender patrones más sutiles en los datos y la capacidad de generalizar a nuevas situaciones.
Exploración y Explotación Balanceadas: Es posible que este agente haya logrado un mejor equilibrio entre la exploración (aprender de nuevas experiencias) y
la explotación (aplicar el conocimiento existente para maximizar la recompensa) durante el entrenamiento y la prueba.
Adaptación al Entorno: La arquitectura compleja puede haber permitido al agente adaptarse mejor a las condiciones cambiantes del mercado y aprender estrategias más sofisticadas para maximizar el profit.


En resumen, las diferencias en los profits entre los agentes con arquitecturas simples y complejas pueden atribuirse a una variedad de factores, incluyendo la capacidad de aprendizaje,
la generalización, la exploración adecuada del espacio de acciones y la adaptación al entorno. Cada agente puede haber encontrado un equilibrio diferente entre estos factores,
lo que resulta en diferentes niveles de éxito en la tarea.


Gracias
César

"""

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Recuerda poner en esta sección la ruta en la que guardaste los archivos correspondientes a los modelos que generamos en la misión anterior.

In [6]:
#%cd "drive/MyDrive/RL_/"
%cd /content/drive/MyDrive/RL_
!pwd
!ls

/content/drive/MyDrive/RL_
/content/drive/MyDrive/RL_
'Misión 3: Entrenando el agente inteligente.ipynb'


## Recapitulación de la misión anterior

In [44]:
!pip install plotly
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

In [ ]:
#!pip install tensorflow-gpu

In [45]:
# Paquetes básicos para el análisis de datos
import math
import random
import numpy as np
import pandas as pd
import pandas_datareader as data_reader
from collections import deque

# Paquetes especializados en graficación e interacción
%matplotlib notebook
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Paquete especializado en tensores y, sobre todo, en redes neuronales
import tensorflow as tf
from tensorflow import keras

In [ ]:
def dataset_loader(stock_name, clip_data, train_portion=0.5):
    full_dataset = data_reader.DataReader(
        stock_name, data_source="yahoo").iloc[-clip_data:]

    data_start_date = str(full_dataset.index[0])
    data_end_date = str(full_dataset.index[-1])

    data = full_dataset['Close']
    data_samples = len(data)
    data_mid_date = str(full_dataset.index[int(data_samples * train_portion)])

    train_data = data[:data_mid_date]
    test_data = data[data_mid_date:]

    fmt_str = "{} <---[{}]---> {} <---[{}]---> {}"
    print("Timeline:\n", fmt_str.format(
        data_start_date.split()[0], len(train_data),
        data_mid_date.split()[0], len(test_data),
        data_end_date.split()[0]))

    return train_data, test_data

In [46]:
!pip install alpha_vantage


In [4]:
#KOV4S8MT1CZK6MB5

# Importar bibliotecas necesarias
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime

# Definir función para cargar el conjunto de datos
def dataset_loader(stock_name, clip_data, train_portion=0.5):
    # Crear una instancia de TimeSeries con la API key
    ts = TimeSeries(key='KOV4S8MT1CZK6MB5', output_format='pandas')

    # Obtener los datos diarios de precios de las acciones
    data, meta_data = ts.get_daily(symbol=stock_name, outputsize='full')

    # Obtener el DataFrame con los datos
    df = data.iloc[-clip_data:]

    # Ordenar el índice del DataFrame por fecha
    df.sort_index(inplace=True)

    # Obtener las fechas de inicio y fin
    data_start_date = df.index[0].strftime('%Y-%m-%d')
    data_end_date = df.index[-1].strftime('%Y-%m-%d')

    # Obtener la fecha media para dividir los datos de entrenamiento y prueba
    data_samples = len(df)
    data_mid_date = df.index[int(data_samples * train_portion)].strftime('%Y-%m-%d')

    # Dividir los datos en datos de entrenamiento y prueba
    train_data = df.loc[:data_mid_date, '4. close']
    test_data = df.loc[data_mid_date:, '4. close']

    # Imprimir la línea de tiempo
    fmt_str = "{} <---[{}]---> {} <---[{}]---> {}"
    print("Timeline:\n", fmt_str.format(
        data_start_date, len(train_data),
        data_mid_date, len(test_data),
        data_end_date))

    return train_data, test_data

In [47]:
class Trading_Environment():
    def __init__(self, dataset, verbose=False):
        self.window_size = 11
        self.state_size = self.window_size
        self.action_space = {0: "Stay", 1: "Buy", 2: "Sell"}

        self.data = dataset
        self.data_length = len(dataset)

        self.inventory = None
        self.current_step = None

        self.verbose = verbose

        self.reset()

    def reset(self):
        self.inventory = []
        self.current_step = 0

        return self._get_state()

    def step(self, action):
        reward = 0  # Only increases when selling is positive
        profit = 0

        current_price = self.data[self.current_step]

        if action == 1: # Buying
            self.inventory.append(current_price)

            if self.verbose:
                print("Robot-RL {:6s}: {}".format(
                    "Compró", self.stock_price_format(current_price)),
                    "Inventario: ", len(self.inventory))

        elif action == 2 and len(self.inventory) > 0: # Selling
            buy_price = self.inventory.pop(0)

            difference = current_price - buy_price

            reward = difference  # max(difference, 0)
            profit = difference
            if self.verbose:
                print("Robot-RL {:6s}: {}".format(
                    "Vendió", self.stock_price_format(current_price)),
                    "Inventario: ", len(self.inventory))

        else: # Staying
            if self.verbose:
                print("Robot-RL {:6s}: {}".format(
                    "Esperó", self.stock_price_format(current_price)),
                    "Inventario: ", len(self.inventory))

        self.current_step += 1
        next_state = self._get_state()

        if self.current_step >= self.data_length - 1:
            done = True
        else:
            done = False

        return next_state, reward, done, profit

    def _get_state(self):
         # Calcular el ID de inicio
        starting_id = self.current_step - self.window_size + 1

        # Datos en ventana
        if starting_id >= 0:
            windowed_data = self.data[starting_id : self.current_step + 1].tolist()
        else:
            windowed_data = -starting_id * [self.data[0]] + \
                            list(self.data[0 : self.current_step + 1])

        # Normalizar los datos de precio con la función sigmoidal
        state = 1. / (1. + np.exp(-np.diff(windowed_data)))

        # Analizar la modificación de los estados del entorno:
        # Aquí, calculamos la diferencia entre puntos de datos consecutivos y los normalizamos usando una función sigmoidal.
        # Esto captura la tendencia en los movimientos de precios dentro de los datos de la ventana.

        # Devolver la observación / estado
        return np.concatenate(([len(self.inventory) * self.current_value], state))

    def get_action_sample(self):
        return random.randrange(len(self.action_space))

    @property
    def current_value(self):
        return self.data[self.current_step]

    @staticmethod
    def stock_price_format(n):
        return "- $ {:10f}, ".format(abs(n)) if n < 0 else "+ $ {:10f},".format(abs(n))


In [92]:
class DQL_Agent():
    def __init__(self, environment, agent_name="Q_model", pretrained=False):
        self.env = environment
        self.memory = deque(maxlen=2000)
        self.batch_size = 32
        self.agent_name = agent_name

        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        self.first_step = True


        """ ajuste -------------------------------------------------------------

        Se han creado dos modelos: model y target_model.
        Durante la experiencia de repetición, se utiliza target_model para obtener las acciones óptimas para los siguientes estados.
        Se utilizan los valores Q calculados con target_model para calcular los valores objetivo.
        Se actualizan los valores Q actuales utilizando los valores objetivo y se entrena el modelo model.
        --- ajuste -------------------------------------------------------------  """

        # Creamos dos modelos: model y target_model
        self.model = self.create_model()
        self.target_model = self.create_model()


        #self.model = self.load_model() if pretrained else self.create_model()

        # Aquí se puede definir el método para crear la arquitectura intermedia
        #self.model = self.load_model() if pretrained else self.create_intermediate_model()

        # Aquí se puede definir el método para crear la arquitectura compleja
        self.model = self.load_model() if pretrained else self.create_complex_model()

        # Aquí se puede definir el método para crear la arquitectura simple
        #self.model = self.load_model() if pretrained else self.create_simple_model()

        """
        Ahora puedes ejecutar los experimentos y comparar los resultados entre las tres arquitecturas diferentes.
        """

    # Definir método para crear un modelo con arquitectura intermedia
    def create_intermediate_model(self):
        model = keras.models.Sequential(name=self.agent_name + "_Intermediate")

        model.add(keras.Input(shape=(self.env.state_size,)))
        model.add(keras.layers.Dense(name="Layer1", units=96, activation='relu'))
        model.add(keras.layers.Dense(name="Layer2", units=192, activation='relu'))
        model.add(keras.layers.Dense(name="OutputLayer", units=len(self.env.action_space), activation='linear'))

        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    # Método para crear un modelo con arquitectura compleja
    def create_complex_model(self):
        model = keras.models.Sequential(name=self.agent_name + "_Complex")

        model.add(keras.Input(shape=(self.env.state_size,)))
        model.add(keras.layers.Dense(name="Layer1", units=128, activation='relu'))
        model.add(keras.layers.Dense(name="Layer2", units=256, activation='relu'))
        model.add(keras.layers.Dense(name="Layer3", units=256, activation='relu'))
        model.add(keras.layers.Dense(name="OutputLayer", units=len(self.env.action_space), activation='linear'))

        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    # Método para crear un modelo con arquitectura sencilla
    def create_simple_model(self):
        model = keras.models.Sequential(name=self.agent_name + "_Simple")

        model.add(keras.Input(shape=(self.env.state_size,)))
        model.add(keras.layers.Dense(name="Layer1", units=64, activation='relu'))
        model.add(keras.layers.Dense(name="OutputLayer", units=len(self.env.action_space), activation='linear'))

        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model


    def create_model(self):
        model = keras.models.Sequential(name=self.agent_name)

        model.add(keras.Input(shape=(self.env.state_size,)))
        #model.add(keras.layers.Dense(name="Layer1", units=128, activation='relu'))
        #model.add(keras.layers.Dense(name="Layer2", units=256, activation='relu'))
        #model.add(keras.layers.Dense(name="Layer3", units=256, activation='relu'))
        model.add(keras.layers.Dense(name="Layer4", units=32, activation='relu'))
        model.add(keras.layers.Dense(name="Layer5", units=len(self.env.action_space), activation='linear'))

        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def get_action(self, s, is_evaluating=False):
        if self.first_step:
            self.first_step = False
            return 1

        if not is_evaluating and random.random() <= self.epsilon:
            return self.env.get_action_sample()

        else:
            return int(tf.argmax(self.model.predict(
                s.reshape(1, self.env.state_size), verbose=0)[0]))

    def remember(self, sarsd):
        # sarsd: (state, action, reward, new_state, done)
        self.memory.append(sarsd)

    def experience_replay(self):
        batch_sample = random.sample(self.memory, self.batch_size)

        states = np.array([sarsd[0] for sarsd in batch_sample])
        actions = np.array([sarsd[1] for sarsd in batch_sample])
        rewards = np.array([sarsd[2] for sarsd in batch_sample])
        next_states = np.array([sarsd[3] for sarsd in batch_sample])
        done = np.array([sarsd[4] for sarsd in batch_sample])

        """
        """

        """ ajuste -------------------------------------------------------------

        # Utilizar el primer modelo para obtener las acciones óptimas para los siguientes estados
        best_actions_next = np.argmax(self.model.predict(next_states, verbose=0), axis=1)

        --- ajuste -------------------------------------------------------------  """


        # Utilizar el segundo modelo (target_model) para obtener las acciones óptimas para los siguientes estados
        best_actions_next = np.argmax(self.target_model.predict(next_states, verbose=0), axis=1)



        # Calcular los valores Q para los siguientes estados utilizando el segundo modelo y las acciones óptimas obtenidas anteriormente
        q_values_next = self.target_model.predict(next_states, verbose=0)


        """

        ajuste -------------------------------------------------------------

        Se han creado dos modelos: model y target_model.
        Durante la experiencia de repetición, se utiliza target_model para obtener las acciones óptimas para los siguientes estados.
        Se utilizan los valores Q calculados con target_model para calcular los valores objetivo.
        Se actualizan los valores Q actuales utilizando los valores objetivo y se entrena el modelo model.

        --- ajuste -------------------------------------------------------------  """

        # Calcular los valores Q para los estados actuales utilizando el primer modelo
        q_values = self.model.predict(states, verbose=0)

        # Calcular los valores objetivo utilizando los valores Q obtenidos del segundo modelo y las acciones óptimas obtenidas anteriormente
        targets = rewards + self.gamma * q_values_next[np.arange(self.batch_size), best_actions_next]

        # Actualizar los valores Q actuales utilizando los valores objetivo
        q_values[np.arange(self.batch_size), actions] = targets


        # Q predicho
        #target = rewards + self.gamma * np.amax(self.model.predict(next_states, verbose=0), axis=1)
        #target[done] = rewards[done]

        # Q actual
        #q_values = self.model.predict(states, verbose=0)
        # q_values[range(self.batch_size), actions] = target

        loss = self.model.fit(states, q_values, epochs=1, verbose=0).history["loss"][0]

        self.update_epsilon()

        return loss

    def update_epsilon(self):
        if self.epsilon >= self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save_model(self, episode):
        self.model.save("models/{}_{}".format(self.agent_name, episode))

    def load_model(self):
        return keras.models.load_model("models/" + self.agent_name)

    def save_model(self, episode):
        # Verificar si el directorio existe, si no, crearlo
        save_dir = '/content/drive/MyDrive/RL_'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Guardar el modelo en el directorio especificado
        save_path = os.path.join(save_dir, "{}_{}.h5".format(self.agent_name, episode))
        self.model.save(save_path)

In [93]:
def train_model(agent, env, num_episodes=10):
    profit_evolution = []
    reward_evolution = []
    average_loss = []

    for episode in range(1, num_episodes + 1):

        total_profit = 0
        total_reward = 0
        loss_list = []

        state = env.reset()
        done = False
        while not done:
            action = agent.get_action(state)
            new_state, reward, done, profit = env.step(action)

            sarsd = (state, action, reward, new_state, done)
            agent.remember(sarsd)

            if len(agent.memory) > agent.batch_size:
                loss = agent.experience_replay()
                loss_list.append(loss)

            state = new_state

            total_profit += profit
            total_reward += reward

            #print("Step: {}, Loss: {}, Profit: {}".format(env.current_step, loss, profit))

        print(f"Episode {episode}, Profit: {total_profit}, Reward: {total_reward}")

        profit_evolution.append(total_profit)
        reward_evolution.append(total_reward)
        if len(loss_list) > 0:
            average_loss.append(np.mean(np.array(loss_list)))

        if episode % 10 == 0:
            agent.save_model(f"{episode}")

    agent.save_model(f"{num_episodes}")
    return profit_evolution, reward_evolution, average_loss

## Ejercicio 1


In [94]:
def test_model(agent, env, epsilon=None):
    profit_evolution = []
    reward_evolution = []
    history = []

    total_profit = 0
    total_reward = 0

    if epsilon is None:
        evaluation_flag = True
    else:
        evaluation_flag = False
        agent.epsilon_decay = 0.0
        agent.epsilon_min = 0.6

    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state, is_evaluating=evaluation_flag)
        new_state, reward, done, profit = env.step(action)

        sarsd = (state, action, reward, new_state, done)
        agent.remember(sarsd)

        state = new_state

        history.append((env.current_value, env.action_space[action]))

        total_profit += profit
        profit_evolution.append(total_profit)
        reward_evolution.append(reward)

        #print(f"Step {env.current_step}, Profit: {profit}, Reward: {reward}")
        #print("Step: {}, Profit: {}".format(env.current_step, profit))

    return profit_evolution, reward_evolution, history


In [51]:
train_data, test_data = dataset_loader(
    stock_name="AAPL", clip_data=200, train_portion=0.5)

# Experimento 1
entorno_exp1= Trading_Environment(test_data)
dql_agent_exp1 = DQL_Agent(entorno_exp1, agent_name="robot_dql_exp1", pretrained=False)  # Usar arquitectura simple

Timeline:
 1999-11-01 <---[101]---> 2000-03-24 <---[100]---> 2000-08-15


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [84]:
"""
dql_agent_exp1 con una arquitectura simple, mostro mejores resultados que el resto de agentes, ahora revisaremos dql_agent_exp1 con una arquitecura intermedia
"""

train_data, test_data = dataset_loader(
    stock_name="AAPL", clip_data=200, train_portion=0.5)

# Experimento agente dql_agent_exp1 arquitectura intermedia
entorno_exp1= Trading_Environment(test_data)
dql_agent_exp1 = DQL_Agent(entorno_exp1, agent_name="robot_dql_exp1", pretrained=False)  # Usar arquitectura intermedia

Timeline:
 1999-11-01 <---[101]---> 2000-03-24 <---[100]---> 2000-08-15


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [95]:
"""
dql_agent_exp1 con una arquitectura simple, mostro mejores resultados que el resto de agentes, ahora revisaremos dql_agent_exp1 con una arquitecura compleja
"""

train_data, test_data = dataset_loader(
    stock_name="AAPL", clip_data=200, train_portion=0.5)

# Experimento agente dql_agent_exp1 arquitectura compleja
entorno_exp1= Trading_Environment(test_data)
dql_agent_exp1 = DQL_Agent(entorno_exp1, agent_name="robot_dql_exp1", pretrained=False)  # Usar arquitectura compleja

Timeline:
 1999-11-01 <---[101]---> 2000-03-24 <---[100]---> 2000-08-15


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [52]:
train_data, test_data = dataset_loader(
    stock_name="AAPL", clip_data=200, train_portion=0.5)

# Experimento 2
entorno_exp2= Trading_Environment(test_data)
dql_agent_exp2 = DQL_Agent(entorno_exp2, agent_name="robot_dql_exp2", pretrained=False)  # Usar arquitectura intermedia

Timeline:
 1999-11-01 <---[101]---> 2000-03-24 <---[100]---> 2000-08-15


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [53]:
train_data, test_data = dataset_loader(
    stock_name="AAPL", clip_data=200, train_portion=0.5)

# Experimento 3
entorno_exp3 = Trading_Environment(test_data)
dql_agent_exp3 = DQL_Agent(entorno_exp3, agent_name="robot_dql_exp3", pretrained=False)  # Usar arquitectura compleja

Timeline:
 1999-11-01 <---[101]---> 2000-03-24 <---[100]---> 2000-08-15


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [49]:
train_data, test_data = dataset_loader(
    stock_name="AAPL", clip_data=200, train_portion=0.5)

entorno = Trading_Environment(test_data)
dql_agent = DQL_Agent(entorno, agent_name="robot_d2ql_FINAL", pretrained=False) # True

Timeline:
 1999-11-01 <---[101]---> 2000-03-24 <---[100]---> 2000-08-15


<ipython-input-12-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Prueba con una función greedy

In [54]:
pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=None)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [85]:
"""
Seguimiento adql_agent_exp1 con una arquitecura intermedia

"""

pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=None)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [96]:
"""
Seguimiento adql_agent_exp1 con una arquitecura compleja

"""
pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=None)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [55]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()

In [86]:
"""
Seguimiento adql_agent_exp1 con una arquitecura intermedia

"""


x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()

In [97]:
"""
Seguimiento adql_agent_exp1 con una arquitecura compleja

"""


x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()

In [56]:
pev, rev, history = test_model(agent=dql_agent_exp2, env=entorno_exp2, epsilon=None)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [57]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()

In [58]:
pev, rev, history = test_model(agent=dql_agent_exp3, env=entorno_exp3, epsilon=None)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [59]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()


In [50]:
pev, rev, history = test_model(agent=dql_agent, env=entorno, epsilon=None)

In [33]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()



In [51]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()


## Ejercicio 2

*Prueba* con una función 0.1-greedy

In [60]:
pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [87]:
"""
Seguimiento adql_agent_exp1 con una arquitecura intermedia

"""
pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=0.01)


<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [98]:
"""
Seguimiento adql_agent_exp1 con una arquitecura compleja

"""

pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=0.01)


<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [61]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()


In [88]:
"""
Seguimiento adql_agent_exp1 con una arquitecura intermedia

"""

x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()

In [99]:
"""
Seguimiento adql_agent_exp1 con una arquitecura compleja

"""

x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()

In [62]:
pev, rev, history = test_model(agent=dql_agent_exp2, env=entorno_exp2, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [63]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()


In [64]:
pev, rev, history = test_model(agent=dql_agent_exp2, env=entorno_exp2, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [65]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()

In [66]:
pev, rev, history = test_model(agent=dql_agent_exp3, env=entorno_exp3, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [67]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()



In [52]:
pev, rev, history = test_model(agent=dql_agent, env=entorno, epsilon=0.01)

In [53]:
x_list = np.arange(len(pev))

fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)


data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()



## Ejercicio 3

In [68]:
"""
Este código carga datos de prueba para el stock "AAPL", recorta los datos a los primeros 1000 puntos y divide los datos de entrenamiento y prueba en una proporción de 50-50.
Luego, para el Experimento 1, se crea un entorno de trading utilizando estos datos de prueba, con la opción de impresión detallada desactivada.
Además, se inicializa un agente DQL para este experimento, con un nombre específico y sin cargar un modelo preentrenado.
"""

# Cargar los datos del conjunto de pruebas para el stock "AAPL"
# Utiliza solo los primeros 1000 datos y divide los datos de entrenamiento y prueba en una proporción de 0.5
_, test_data = dataset_loader(stock_name="AAPL", clip_data=1000, train_portion=0.5)
# Experimento 1
# Creamos un entorno de trading con los datos de prueba cargados, sin imprimir información detallada (verbose=False)
entorno_exp1 = Trading_Environment(test_data, verbose=False)
# Creamos un agente DQL para el experimento 1, sin cargar un modelo preentrenado (pretrained=False)
dql_agent_exp1 = DQL_Agent(entorno_exp1, agent_name="robot_d5ql_full_50_exp_1", pretrained=False) # true


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Timeline:
 1999-11-01 <---[501]---> 2001-10-30 <---[500]---> 2003-10-23


In [89]:
"""
Seguimiento adql_agent_exp1 con una arquitecura intermedia

"""
# Cargar los datos del conjunto de pruebas para el stock "AAPL"
# Utiliza solo los primeros 1000 datos y divide los datos de entrenamiento y prueba en una proporción de 0.5
_, test_data = dataset_loader(stock_name="AAPL", clip_data=1000, train_portion=0.5)
# Experimento 1
# Creamos un entorno de trading con los datos de prueba cargados, sin imprimir información detallada (verbose=False)
entorno_exp1 = Trading_Environment(test_data, verbose=False)
# Creamos un agente DQL para el experimento 1, sin cargar un modelo preentrenado (pretrained=False)
dql_agent_exp1 = DQL_Agent(entorno_exp1, agent_name="robot_d5ql_full_50_exp_1", pretrained=False) # true


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Timeline:
 1999-11-01 <---[501]---> 2001-10-30 <---[500]---> 2003-10-23


In [100]:
"""
Seguimiento adql_agent_exp1 con una arquitecura compleja

"""

# Cargar los datos del conjunto de pruebas para el stock "AAPL"
# Utiliza solo los primeros 1000 datos y divide los datos de entrenamiento y prueba en una proporción de 0.5
_, test_data = dataset_loader(stock_name="AAPL", clip_data=1000, train_portion=0.5)
# Experimento 1
# Creamos un entorno de trading con los datos de prueba cargados, sin imprimir información detallada (verbose=False)
entorno_exp1 = Trading_Environment(test_data, verbose=False)
# Creamos un agente DQL para el experimento 1, sin cargar un modelo preentrenado (pretrained=False)
dql_agent_exp1 = DQL_Agent(entorno_exp1, agent_name="robot_d5ql_full_50_exp_1", pretrained=False) # true

Timeline:
 1999-11-01 <---[501]---> 2001-10-30 <---[500]---> 2003-10-23


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [69]:
dql_agent_exp1.model.summary()

Model: "robot_d5ql_full_50_exp_1_Simple"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 64)                768       
                                                                 
 OutputLayer (Dense)         (None, 3)                 195       
                                                                 
Total params: 963 (3.76 KB)
Trainable params: 963 (3.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
"""
Carga los datos de prueba para el stock "AAPL", recorta los datos a los primeros 1000 puntos y divide los datos de entrenamiento y prueba en una proporción de 50-50.
Luego, para el Experimento 2, se crea un entorno de trading utilizando estos datos de prueba, con la opción de impresión detallada desactivada.
Además, se inicializa un agente DQL para este experimento, con un nombre específico y sin cargar un modelo preentrenado.
"""

# Cargar los datos del conjunto de pruebas para el stock "AAPL"
# Utiliza solo los primeros 1000 datos y divide los datos de entrenamiento y prueba en una proporción de 0.5
_, test_data = dataset_loader(stock_name="AAPL", clip_data=1000, train_portion=0.5)
# Experimento 2
# Creamos un entorno de trading con los datos de prueba cargados, sin imprimir información detallada (verbose=False)
entorno_exp2 = Trading_Environment(test_data, verbose=False)

# Creamos un agente DQL para el experimento 2, sin cargar un modelo preentrenado (pretrained=False)
dql_agent_exp2 = DQL_Agent(entorno_exp2, agent_name="robot_d5ql_full_50_exp_2", pretrained=False) # true


Timeline:
 1999-11-01 <---[501]---> 2001-10-30 <---[500]---> 2003-10-23


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [71]:
dql_agent_exp2.model.summary()

Model: "robot_d5ql_full_50_exp_2_Simple"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 64)                768       
                                                                 
 OutputLayer (Dense)         (None, 3)                 195       
                                                                 
Total params: 963 (3.76 KB)
Trainable params: 963 (3.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
"""
En este bloque de código, se carga el conjunto de datos de prueba para el stock "AAPL",
se recortan los datos a los primeros 1000 puntos y se dividen en datos de entrenamiento y prueba en una proporción de 50-50.
Luego, para el Experimento 3, se crea un entorno de trading utilizando estos datos de prueba, con la opción de impresión detallada desactivada.
Además, se inicializa un agente DQL para este experimento, con un nombre específico y sin cargar un modelo preentrenado.

"""

# Cargar los datos del conjunto de pruebas para el stock "AAPL"
# Utiliza solo los primeros 1000 datos y divide los datos de entrenamiento y prueba en una proporción de 0.5
_, test_data = dataset_loader(stock_name="AAPL", clip_data=1000, train_portion=0.5)
# Experimento 3
# Creamos un entorno de trading con los datos de prueba cargados, sin imprimir información detallada (verbose=False)
entorno_exp3 = Trading_Environment(test_data, verbose=False)
# Creamos un agente DQL para el experimento 3, sin cargar un modelo preentrenado (pretrained=False)
dql_agent_exp3 = DQL_Agent(entorno_exp3, agent_name="robot_d5ql_full_50_exp_3", pretrained=False) # true


Timeline:
 1999-11-01 <---[501]---> 2001-10-30 <---[500]---> 2003-10-23


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [73]:
dql_agent_exp3.model.summary()

Model: "robot_d5ql_full_50_exp_3_Simple"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 64)                768       
                                                                 
 OutputLayer (Dense)         (None, 3)                 195       
                                                                 
Total params: 963 (3.76 KB)
Trainable params: 963 (3.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
_, test_data = dataset_loader(stock_name="AAPL", clip_data=1000, train_portion=0.5)

entorno = Trading_Environment(test_data, verbose=False)
agente = DQL_Agent(entorno, agent_name="robot_d5ql_full_50", pretrained=False) # true


Timeline:
 1999-11-01 <---[501]---> 2001-10-30 <---[500]---> 2003-10-23


<ipython-input-4-3669e1f5687e>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-bdf297553138>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-5-bdf297553138>:87: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [33]:
agente.model.summary()

Model: "robot_d5ql_full_50_Simple"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 64)                768       
                                                                 
 OutputLayer (Dense)         (None, 3)                 195       
                                                                 
Total params: 963 (3.76 KB)
Trainable params: 963 (3.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
!pip install visualkeras
import visualkeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.8/993.8 kB 12.6 MB/s eta 0:00:00


In [35]:
visualkeras.layered_view(dql_agent_exp1.model)

In [36]:
visualkeras.layered_view(dql_agent_exp2.model)

In [37]:
visualkeras.layered_view(dql_agent_exp3.model)

In [57]:
visualkeras.layered_view(agente.model)

In [75]:

# Ejecutar el modelo entrenado en el entorno de prueba
# para evaluar su desempeño.
# Utiliza el agente DQL del Experimento 1 (dql_agent_exp1)
# y el entorno correspondiente (entorno_exp1).
# Establece el valor de epsilon en 0.01 para reducir la exploración.

pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [90]:
"""
Seguimiento adql_agent_exp1 con una arquitecura intermedia

"""

pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [101]:
"""
Seguimiento adql_agent_exp1 con una arquitecura compleja

"""

pev, rev, history = test_model(agent=dql_agent_exp1, env=entorno_exp1, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [76]:
# Generar un gráfico para visualizar la evolución de la ganancia (Profit Evolution) y la recompensa (Reward)
# a lo largo de los pasos de tiempo, así como las decisiones de compra, venta y espera tomadas por el agente.
x_list = np.arange(len(pev))

# Crear un objeto de trazado con dos filas y una columna para mostrar los gráficos de Profit Evolution y Reward
fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Agregar la evolución de la ganancia (pev) como una línea continua en el primer gráfico
fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
# Agregar la recompensa (rev) como una línea continua con menor opacidad en el primer gráfico
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)

# Extraer los datos de la observación del entorno (historia de observaciones) y las acciones tomadas por el agente
data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

# Identificar los índices de las acciones de "Stay", "Buy" y "Sell"
stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

# Agregar el dataset como una línea continua en el segundo gráfico
fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

# Actualizar los ejes x con el título "Step"
fig1.update_xaxes(title_text="Step")

# Mostrar el gráfico
fig1.show()

"""
Este bloque de código genera un gráfico que muestra la evolución de la ganancia y la recompensa a lo largo de los pasos de tiempo,
así como las decisiones de compra, venta y espera tomadas por el agente durante el experimento.
Las líneas continuas representan la evolución de la ganancia y el dataset, mientras que los puntos indican las acciones específicas tomadas por el agente.

"""

In [91]:
"""
Seguimiento adql_agent_exp1 con una arquitecura intermedia

"""

# Generar un gráfico para visualizar la evolución de la ganancia (Profit Evolution) y la recompensa (Reward)
# a lo largo de los pasos de tiempo, así como las decisiones de compra, venta y espera tomadas por el agente.
x_list = np.arange(len(pev))

# Crear un objeto de trazado con dos filas y una columna para mostrar los gráficos de Profit Evolution y Reward
fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Agregar la evolución de la ganancia (pev) como una línea continua en el primer gráfico
fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
# Agregar la recompensa (rev) como una línea continua con menor opacidad en el primer gráfico
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)

# Extraer los datos de la observación del entorno (historia de observaciones) y las acciones tomadas por el agente
data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

# Identificar los índices de las acciones de "Stay", "Buy" y "Sell"
stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

# Agregar el dataset como una línea continua en el segundo gráfico
fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

# Actualizar los ejes x con el título "Step"
fig1.update_xaxes(title_text="Step")

# Mostrar el gráfico
fig1.show()

"""
Este bloque de código genera un gráfico que muestra la evolución de la ganancia y la recompensa a lo largo de los pasos de tiempo,
así como las decisiones de compra, venta y espera tomadas por el agente durante el experimento.
Las líneas continuas representan la evolución de la ganancia y el dataset, mientras que los puntos indican las acciones específicas tomadas por el agente.

"""



'\nEste bloque de código genera un gráfico que muestra la evolución de la ganancia y la recompensa a lo largo de los pasos de tiempo, \nasí como las decisiones de compra, venta y espera tomadas por el agente durante el experimento. \nLas líneas continuas representan la evolución de la ganancia y el dataset, mientras que los puntos indican las acciones específicas tomadas por el agente.\n\n'

In [102]:
"""
Seguimiento adql_agent_exp1 con una arquitecura compleja

"""

# Generar un gráfico para visualizar la evolución de la ganancia (Profit Evolution) y la recompensa (Reward)
# a lo largo de los pasos de tiempo, así como las decisiones de compra, venta y espera tomadas por el agente.
x_list = np.arange(len(pev))

# Crear un objeto de trazado con dos filas y una columna para mostrar los gráficos de Profit Evolution y Reward
fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Agregar la evolución de la ganancia (pev) como una línea continua en el primer gráfico
fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
# Agregar la recompensa (rev) como una línea continua con menor opacidad en el primer gráfico
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)

# Extraer los datos de la observación del entorno (historia de observaciones) y las acciones tomadas por el agente
data_array = np.array([his[0] for his in history])
action_list = np.array([his[1] for his in history])

# Identificar los índices de las acciones de "Stay", "Buy" y "Sell"
stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

# Agregar el dataset como una línea continua en el segundo gráfico
fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

# Actualizar los ejes x con el título "Step"
fig1.update_xaxes(title_text="Step")

# Mostrar el gráfico
fig1.show()

"""
Este bloque de código genera un gráfico que muestra la evolución de la ganancia y la recompensa a lo largo de los pasos de tiempo,
así como las decisiones de compra, venta y espera tomadas por el agente durante el experimento.
Las líneas continuas representan la evolución de la ganancia y el dataset, mientras que los puntos indican las acciones específicas tomadas por el agente.

"""


'\nEste bloque de código genera un gráfico que muestra la evolución de la ganancia y la recompensa a lo largo de los pasos de tiempo, \nasí como las decisiones de compra, venta y espera tomadas por el agente durante el experimento. \nLas líneas continuas representan la evolución de la ganancia y el dataset, mientras que los puntos indican las acciones específicas tomadas por el agente.\n\n'

In [77]:
# Realiza pruebas del modelo entrenado con el agente del Experimento 2 en el entorno del Experimento 2
# con un valor de epsilon de 0.01 para la exploración epsilon-greedy
pev, rev, history = test_model(agent=dql_agent_exp2, env=entorno_exp2, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [78]:
# Crea una lista de índices para el eje x basada en la longitud de la lista de Profit Evolution (pev)
x_list = np.arange(len(pev))

# Crea una figura con dos subplots, compartiendo el eje x
fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Añade una traza para visualizar la evolución de la ganancia (Profit Evolution) en el primer subplot
fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
# Añade una traza para visualizar la recompensa (Reward) en el primer subplot
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)

# Crea un array de numpy con los datos de la historia de acciones tomadas por el agente durante las pruebas
data_array = np.array([his[0] for his in history])

# Crea un array de numpy con las acciones tomadas por el agente durante las pruebas
action_list = np.array([his[1] for his in history])

# Filtra los índices donde el agente decidió 'quedarse' (Stay), 'comprar' (Buy) y 'vender' (Sell)
stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

# Añade una traza para visualizar el conjunto de datos en el segundo subplot
fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
# Añade trazas para visualizar los puntos donde el agente decidió quedarse, comprar y vender en el segundo subplot
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

# Actualiza el título del eje x
fig1.update_xaxes(title_text="Step")
# Muestra la figura
fig1.show()


"""

Este bloque de código crea un gráfico con dos subplots.
En el primer subplot se muestra la evolución de la ganancia (Profit Evolution) y la recompensa (Reward) a lo largo del tiempo.
En el segundo subplot se muestra el conjunto de datos y los puntos donde el agente decidió quedarse, comprar y vender durante las pruebas.


"""

In [79]:

# Realiza pruebas del modelo entrenado en el tercer experimento (dql_agent_exp3) en el entorno del tercer experimento (entorno_exp3)
# con un valor de epsilon de 0.01
pev, rev, history = test_model(agent=dql_agent_exp3, env=entorno_exp3, epsilon=0.01)

<ipython-input-47-04752e11e2c3>:73: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:91: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-47-04752e11e2c3>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [80]:
# Crea una lista de valores de x para el gráfico basada en la longitud de la serie de datos de la evolución de beneficios (pev)
x_list = np.arange(len(pev))

# Crea una figura con dos subgráficos (rows=2, cols=1) con ejes x compartidos (shared_xaxes=True)
fig1 = make_subplots(rows=2, cols=1, shared_xaxes=True)

# Añade un gráfico de línea para la evolución de los beneficios (pev) al primer subgráfico
fig1.add_trace(go.Scatter(x=x_list, y=pev, mode='lines', name="Profit Evolution",
                          opacity=1), row=1, col=1)
# Añade un gráfico de línea para las recompensas (rev) al primer subgráfico
fig1.add_trace(go.Scatter(x=x_list, y=rev, mode='lines', name="Reward",
                          opacity=0.5), row=1, col=1)

# Convierte la lista de tuplas en un arreglo numpy para facilitar el acceso a los datos
data_array = np.array([his[0] for his in history])
# Convierte la lista de tuplas en un arreglo numpy para facilitar el acceso a las acciones realizadas
action_list = np.array([his[1] for his in history])

# Filtra los índices
stay_indices = action_list == 'Stay'
buy_indices = action_list == 'Buy'
sell_indices = action_list == 'Sell'

# Añade una serie de líneas para mostrar el conjunto de datos al segundo subgráfico
fig1.add_trace(go.Scatter(x=x_list, y=data_array, name="Dataset", mode="lines",
                          line=dict(color="black")), row=2, col=1)
# Añade una serie de puntos para mostrar las posiciones donde la acción fue 'Stay' al segundo subgráfico
fig1.add_trace(go.Scatter(x=x_list[stay_indices], y=data_array[stay_indices],
                          mode='markers', name="stay", opacity=1.0), row=2, col=1)
# Añade una serie de puntos para mostrar las posiciones donde la acción fue 'Buy' al segundo subgráfico
fig1.add_trace(go.Scatter(x=x_list[buy_indices], y=data_array[buy_indices],
                          mode='markers', name="buy", opacity=1.0), row=2, col=1)
# Añade una serie de puntos para mostrar las posiciones donde la acción fue 'Sell' al segundo subgráfico
fig1.add_trace(go.Scatter(x=x_list[sell_indices], y=data_array[sell_indices],
                          mode='markers', name="sell", opacity=1.0), row=2, col=1)

fig1.update_xaxes(title_text="Step")
fig1.show()


"""
Este bloque de código genera un gráfico con dos subgráficos.
En el primer subgráfico se muestra la evolución de los beneficios (Profit Evolution) y las recompensas (Reward) a lo largo del tiempo.
En el segundo subgráfico se muestra el conjunto de datos junto con los puntos donde el agente decidió quedarse ('Stay'), comprar ('Buy') o vender ('Sell').

"""

In [58]:
pev, rev, history = test_model(agent=agente, env=entorno, epsilon=0.01)